In [1]:
# import json
# import gymnasium as gym
# from gym.spaces import Dict, Box
import numpy as np

In [2]:
# from torch import nn
# from torch.functional import *

In [3]:
# import gymnasium as gym
from stable_baselines3 import PPO
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.env_checker import check_env
from stable_baselines3.common.monitor import Monitor
from envs import BlendEnv, flatten_and_track_mappings, reconstruct_dict

In [4]:
env = BlendEnv()

In [5]:
env = Monitor(env)

In [6]:
action_sample = {
    'source_blend':{
        's1': {'j1':1, 'j2':1, 'j3':1, 'j4':0}, # From s1 to b1, from s1 to b2 etc...
        's2': {'j1':1, 'j2':1, 'j3':0, 'j4':1},
    },
    
    # 'source_demand':{
    #     's1': {},
    #     's2': {}
    # },
    
    'blend_blend':{
        'j1': {'j5':1, 'j6':0, 'j7':0, 'j8':0},
        'j2': {'j5':0, 'j6':0, 'j7':0, 'j8':0},
        'j3': {'j5':0, 'j6':0, 'j7':0, 'j8':0},
        'j4': {'j5':0, 'j6':0, 'j7':0, 'j8':0},
        'j5': {},
        'j6': {},
        'j7': {},
        'j8': {}
    },
    
    'blend_demand':{
        'j1': {},
        'j2': {},
        'j3': {},
        'j4': {},
        'j5': {'p1':1, 'p2':0},
        'j6': {'p1':0, 'p2':0},
        'j7': {'p1':0, 'p2':0},
        'j8': {'p1':0, 'p2':0}
    },
    
    "tau": {"s1": 10, "s2": 10},
    
    "delta": {"p1": 0, "p2": 0}
}

In [7]:
x, _ = flatten_and_track_mappings(action_sample)

In [8]:
x

array([ 1.,  1.,  1.,  0.,  1.,  1.,  0.,  1.,  1.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,
        0.,  0.,  0.,  0.,  0.,  0., 10., 10.,  0.,  0.], dtype=float32)

In [9]:
env.mapping_act

c:\Users\adame\OneDrive\Bureau\CODE\BlendingRL\blendv\lib\site-packages\gymnasium\core.py:311: UserWarning: WARN: env.mapping_act to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.mapping_act` for environment variables or `env.get_wrapper_attr('mapping_act')` that will search the reminding wrappers.
  logger.warn(


[(0, ['source_blend', 's1', 'j1']),
 (1, ['source_blend', 's1', 'j2']),
 (2, ['source_blend', 's1', 'j3']),
 (3, ['source_blend', 's1', 'j4']),
 (4, ['source_blend', 's2', 'j1']),
 (5, ['source_blend', 's2', 'j2']),
 (6, ['source_blend', 's2', 'j3']),
 (7, ['source_blend', 's2', 'j4']),
 (8, ['blend_blend', 'j1', 'j5']),
 (9, ['blend_blend', 'j1', 'j6']),
 (10, ['blend_blend', 'j1', 'j7']),
 (11, ['blend_blend', 'j1', 'j8']),
 (12, ['blend_blend', 'j2', 'j5']),
 (13, ['blend_blend', 'j2', 'j6']),
 (14, ['blend_blend', 'j2', 'j7']),
 (15, ['blend_blend', 'j2', 'j8']),
 (16, ['blend_blend', 'j3', 'j5']),
 (17, ['blend_blend', 'j3', 'j6']),
 (18, ['blend_blend', 'j3', 'j7']),
 (19, ['blend_blend', 'j3', 'j8']),
 (20, ['blend_blend', 'j4', 'j5']),
 (21, ['blend_blend', 'j4', 'j6']),
 (22, ['blend_blend', 'j4', 'j7']),
 (23, ['blend_blend', 'j4', 'j8']),
 (24, ['blend_demand', 'j5', 'p1']),
 (25, ['blend_demand', 'j5', 'p2']),
 (26, ['blend_demand', 'j6', 'p1']),
 (27, ['blend_demand', 'j6'

In [10]:
model = PPO("MlpPolicy", env, verbose=1, tensorboard_log="./logs", clip_range=0.4, learning_rate=0.025)

Using cpu device
Wrapping the env in a DummyVecEnv.


In [11]:
model.learn(total_timesteps=100000, progress_bar=True)

Logging to ./logs\PPO_20


c:\Users\adame\OneDrive\Bureau\CODE\BlendingRL\blendv\lib\site-packages\rich\live.py:231: UserWarning: install 
"ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

---------------------------------
| rollout/           |          |
|    ep_len_mean     | 6        |
|    ep_rew_mean     | 2.27e+05 |
| time/              |          |
|    fps             | 947      |
|    iterations      | 1        |
|    time_elapsed    | 2        |
|    total_timesteps | 2048     |
---------------------------------
----------------------------------------
| rollout/                |            |
|    ep_len_mean          | 6          |
|    ep_rew_mean          | 3.33e+05   |
| time/                   |            |
|    fps                  | 735        |
|    iterations           | 2          |
|    time_elapsed         | 5          |
|    total_timesteps      | 4096       |
| train/                  |            |
|    approx_kl            | 0.08893031 |
|    clip_fraction        | 0.231      |
|    clip_range           | 0.4        |
|    entropy_loss         | -51.1      |
|    explained_variance   | -3.1e-06   |
|    learning_rate        | 0.025      |
|   

In [28]:
model.save("./models/test_model_6")

In [12]:
episode_rewards = []
obs = env.reset()
obs, obs_dict = obs
episode_reward = 0

In [13]:
action, _ = model.predict(obs, deterministic=True)

In [14]:
obs, reward, done, term, _ = env.step(action)

In [15]:
reconstruct_dict(action, env.mapping_act)

c:\Users\adame\OneDrive\Bureau\CODE\BlendingRL\blendv\lib\site-packages\gymnasium\core.py:311: UserWarning: WARN: env.mapping_act to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.mapping_act` for environment variables or `env.get_wrapper_attr('mapping_act')` that will search the reminding wrappers.
  logger.warn(


{'source_blend': {'s1': {'j1': 0.0, 'j2': 0.0, 'j3': 0.0, 'j4': 0.0},
  's2': {'j1': 0.0, 'j2': 0.0, 'j3': 0.0, 'j4': 0.0}},
 'blend_blend': {'j1': {'j5': 0.26586124,
   'j6': 0.09064908,
   'j7': 0.0,
   'j8': 0.0},
  'j2': {'j5': 0.0, 'j6': 0.2765106, 'j7': 0.15817249, 'j8': 0.0},
  'j3': {'j5': 0.46713272,
   'j6': 0.14449938,
   'j7': 0.10776277,
   'j8': 0.014393185},
  'j4': {'j5': 0.1579068, 'j6': 0.30558264, 'j7': 0.0, 'j8': 0.02218719}},
 'blend_demand': {'j5': {'p1': 0.0, 'p2': 0.0},
  'j6': {'p1': 0.0, 'p2': 0.0},
  'j7': {'p1': 0.0, 'p2': 0.09865692},
  'j8': {'p1': 0.0, 'p2': 0.014574}},
 'tau': {'s1': 0.0, 's2': 0.0},
 'delta': {'p1': 7.374008, 'p2': 3.542297}}

In [16]:
env.sanitize_action(action)

c:\Users\adame\OneDrive\Bureau\CODE\BlendingRL\blendv\lib\site-packages\gymnasium\core.py:311: UserWarning: WARN: env.sanitize_action to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.sanitize_action` for environment variables or `env.get_wrapper_attr('sanitize_action')` that will search the reminding wrappers.
  logger.warn(


IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [26]:
reconstruct_dict(obs, env.mapping_obs)

c:\Users\adame\OneDrive\Bureau\CODE\BlendingRL\blendv\lib\site-packages\gymnasium\core.py:311: UserWarning: WARN: env.mapping_obs to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.mapping_obs` for environment variables or `env.get_wrapper_attr('mapping_obs')` that will search the reminding wrappers.
  logger.warn(


{'sources': {'s1': 0.0, 's2': 0.0},
 'blenders': {'j1': 0.025327858,
  'j2': 0.01945099,
  'j3': 0.0,
  'j4': 0.0,
  'j5': 0.006290628,
  'j6': 0.0123461755,
  'j7': 0.0,
  'j8': 0.0},
 'demands': {'p1': 0.005687058, 'p2': 0.0028227726},
 'properties': {'j1': {'q1': 0.07756889},
  'j2': {'q1': 0.024059026},
  'j3': {'q1': 0.0},
  'j4': {'q1': 0.0},
  'j5': {'q1': 0.09943337},
  'j6': {'q1': 0.027449414},
  'j7': {'q1': 0.0},
  'j8': {'q1': 0.0}},
 'sources_avail': {'s1': 10.0, 's2': 30.0},
 'demands_avail': {'p1': 0.0, 'p2': 0.0}}

In [18]:
reward

-1004.1

In [31]:
reconstruct_dict(obs, env.mapping_obs)

c:\Users\adame\OneDrive\Bureau\CODE\BlendingRL\blendv\lib\site-packages\gymnasium\core.py:311: UserWarning: WARN: env.mapping_obs to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.mapping_obs` for environment variables or `env.get_wrapper_attr('mapping_obs')` that will search the reminding wrappers.
  logger.warn(


{'sources': {'s1': -1.2205675, 's2': 0.04775163},
 'blenders': {'j1': 0.6178459,
  'j2': -0.40120196,
  'j3': 0.8902958,
  'j4': -0.48955035,
  'j5': -0.2332657,
  'j6': 0.17207849,
  'j7': 0.93699574,
  'j8': -0.07253815},
 'demands': {'p1': 0.050955288, 'p2': 0.0},
 'properties': {'j1': {'q1': 0.08798181},
  'j2': {'q1': -0.054309543},
  'j3': {'q1': 0.0076173875},
  'j4': {'q1': 0.1065817},
  'j5': {'q1': -0.3000575},
  'j6': {'q1': -0.41376528},
  'j7': {'q1': 0.14110266},
  'j8': {'q1': 1.1868775}},
 'sources_avail': {'s1': 10.0, 's2': 30.0},
 'demands_avail': {'p1': 0.0, 'p2': 0.0}}

In [ ]:
episode_reward += reward
episode_rewards.append(episode_reward)

In [14]:
def eval_policy(model=model, env=env, n_eval_episodes=10):
    episode_rewards = []
    for _ in range(n_eval_episodes):
        obs = env.reset()
        episode_reward = 0
        done = False
        while not done:
            action, _ = model.predict(obs, deterministic=True)
            obs, reward, done, _ = env.step(action)
            episode_reward += reward
        episode_rewards.append(episode_reward)
    mean_reward = sum(episode_rewards) / n_eval_episodes
    std_reward = np.std(episode_rewards)
    return mean_reward, std_reward

In [ ]:
eval_policy()